In [1]:
import pandas as pd
df = pd.read_csv("item_comments.csv")
df.head()


,evaluation,label
0,用了一段时间，感觉还不错，可以,正面
1,电视非常好，已经是家里的第二台了。第一天下单，第二天就到本地了，可是物流的人说车坏了，一直催...,正面
2,电视比想象中的大好多，画面也很清晰，系统很智能，更多功能还在摸索中,正面
3,不错,正面
4,用了这么多天了，感觉还不错。夏普的牌子还是比较可靠。希望以后比较耐用，现在是考量质量的时候。,正面


In [2]:
import jieba

df["label"] = df["label"].map({"正面": 1, "负面": 0})
words= []
for i,row in  df.iterrows():
    word = jieba.cut(row['evaluation'])
    result = '  '.join(word)
    words.append(result)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\10690\AppData\Local\Temp\jieba.cache
Loading model cost 0.750 seconds.
Prefix dict has been built successfully.


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer() #将文本转换为数值，构成特征向量
X = vect.fit_transform(words)
X = X.toarray()

In [4]:
words_bag = vect.vocabulary_
print(words_bag)

{'一段时间': 656, '感觉': 4113, '不错': 1005, '可以': 2471, '电视': 5870, '非常': 7790, '已经': 3632, '家里': 3398, '第二台': 6260, '第一天': 6242, '下单': 804, '第二天': 6261, '本地': 5090, '可是': 2486, '物流': 5724, '车坏': 7216, '一直': 673, '客服': 3377, '第三天': 6255, '下午': 803, '点才': 5648, '送过来': 7464, '父母': 5698, '年纪': 3706, '买个': 1182, '画面': 5896, '清晰': 5560, '趁着': 7151, '耳朵': 6565, '还好': 7316, '享受': 1298, '几年': 1924, '想象': 4092, '好多': 3204, '系统': 6348, '智能': 4929, '多功能': 2937, '摸索': 4576, '这么': 7343, '多天': 2949, '夏普': 2898, '牌子': 5718, '还是': 7322, '比较': 5345, '可靠': 2501, '希望': 3648, '以后': 1403, '耐用': 6560, '现在': 5774, '考量': 6552, '质量': 7083, '时候': 4848, '速度': 7507, '很快': 3900, '今天': 1352, '流畅': 5507, '一次': 652, '完美': 3336, '购物': 7096, '体验': 1521, '特意': 5740, '打电话': 4248, '回访': 2750, '小哥': 3458, '辛苦': 7250, '东西': 1031, '没用': 5452, '送货': 7458, '保障': 1575, '活动': 5501, '价格': 1419, '问题': 7689, '漂亮': 5599, '反应速度': 2363, '喜欢': 2696, '三台': 737, '这次': 7366, '愉快': 4094, '58': 245, '好大': 3209, '物有所值': 5723, '值得': 1612, '信赖': 159

In [5]:
y = df['label']

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 1)

In [7]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X_train,y_train)

MLPClassifier()

In [8]:
y_pred = mlp.predict(X_test)
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)
print(score)


0.9207459207459208


In [9]:
def preprocess(text):
    # 加载停用词表
    stopwords = pd.read_csv('../stopWords/stop_words.txt', index_col=False, quoting=3, sep="\t", names=['stopword'], encoding='utf-8')
    # 转换为列表
    stopwords_list = stopwords['stopword'].tolist()
    #分词并剔除停用词
    # 分词
    text = jieba.lcut(text)

    # 剔除停用词
    text = [text for text in text if text not in stopwords_list]

    # 以空格连接词语，得到一个字符串
    result = [' '.join(words)]
    # 返回处理后的分词结果
    return result

def text_vectorizer(text):
    # text_list = text.tolist() if isinstance(text, pd.Series) else [text]
    # vectorizer_path = 'static/vectorizer/vectorizer.pkl'  # 替换为之前保存的向量化器文件名
    # vectorizer = joblib.load(vectorizer_path)
    words2vec = vect.fit_transform(text)
    word_list = words2vec.toarray()
    return word_list

comment = input("请输入你对商品的评价：")
# comment = [' '.join(jieba.cut(comment))]
comment = preprocess(comment)
print(comment)
X_try = text_vectorizer(comment)
y_pred = mlp.predict(X_try)
print(y_pred)


['用  了  一段时间  ，  感觉  还  不错  ，  可以 电视  非常  好  ，  已经  是  家里  的  第二台  了  。  第一天  下单  ，  第二天  就  到  本地  了  ，  可是  物流  的  人  说  车坏  了  ，  一直  催  ，  客服  也  帮  着  催  ，  到  第三天  下午  5  点才  送过来  。  父母  年纪  大  了  ，  买个  大  电视  画面  清晰  ，  趁着  耳朵  还好  使  ，  享受  几年  。 电视  比  想象  中  的  大  好多  ，  画面  也  很  清晰  ，  系统  很  智能  ，  更  多功能  还  在  摸索  中 不错 用  了  这么  多天  了  ，  感觉  还  不错  。  夏普  的  牌子  还是  比较  可靠  。  希望  以后  比较  耐用  ，  现在  是  考量  质量  的  时候  。 物流  速度  很快  ，  非常  棒  ，  今天  就  看  了  电视  ，  非常  清晰  ，  非常  流畅  ，  一次  非常  完美  的  购物  体验 非常  好  ，  客服  还  特意  打电话  做  回访 物流  小哥  不错  ，  辛苦  了  ，  东西  还  没用 送货  速度  快  ，  质量  有  保障  ，  活动  价格  挺  好  的  。  希望  用  的  久  ，  不  出  问题  。 非常  漂亮  ，  也  非常  清晰  ，  反应速度  也  快  。 很  不错  家里  都  喜欢  。  。  。  一次  买  了  三台 送货  非常  快  ！  质量  非常  好  ，  这次  购物  非常  愉快  ！  ！ 58  好大  …  …  都  不错  。  看  质量  咯 物流  很快  ，  物有所值  ，  值得  信赖  。  依旧会  关顾  ！  谢谢  商家  ！ 这  价钱  超值  ，  收到  货  马上  装上  看  了  一下  。  很  清晰  式样  也  蛮  好  ！  赞赞  …  … 目前  感受  还  不错 速度  快  、  服务态度  很  好 挺  好 

In [10]:
comment = input("请输入你对商品的评价：")
comment = [' '.join(jieba.cut(comment))]
print(comment)
X_try = vect.transform(comment)
y_pred = mlp.predict(X_try.toarray())
print(y_pred)

['感觉 不行']
[0]
